# Problem 1

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma.
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [41]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [44]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
Canada_data = BeautifulSoup(source, 'lxml')

In [45]:
column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)

In [69]:
content = Canada_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

# clean dataframe 
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1
                                 
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()

,Postalcode,Borough,Neighborhood
0,M1A\n,Not assigned\n,"Not assigned\n, Not assigned\n, Not assigned\n..."
1,M1B\n,Scarborough\n,"Malvern, Rouge, Malvern, Rouge\n, Malvern, Rou..."
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek, Rouge ..."
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill, Guildwood, ..."
4,M1G\n,Scarborough\n,"Woburn, Woburn\n, Woburn\n, Woburn"


In [70]:
df = df.dropna()
empty = 'Not assigned\n'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]

In [71]:
df = df.replace('\n','',regex=True)

In [72]:
df.head()

,Postalcode,Borough,Neighborhood
1,M1B,Scarborough,"Malvern, Rouge, Malvern, Rouge, Malvern, Rouge..."
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek, Rouge ..."
3,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ..."
4,M1G,Scarborough,"Woburn, Woburn, Woburn, Woburn"
5,M1H,Scarborough,"Cedarbrae, Cedarbrae, Cedarbrae, Cedarbrae"


In [73]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postalcode', 'Borough'])
df2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

In [74]:
df2.head(10)

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge, Malvern, Rouge, Malvern, Rouge..."
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek, Rouge ..."
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ..."
3,M1G,Scarborough,"Woburn, Woburn, Woburn, Woburn"
4,M1H,Scarborough,"Cedarbrae, Cedarbrae, Cedarbrae, Cedarbrae"
5,M1J,Scarborough,"Scarborough Village, Scarborough Village, Scar..."
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park, K..."
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge, Golden Mile, ..."
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village Wes..."
9,M1N,Scarborough,"Birch Cliff, Cliffside West, Birch Cliff, Clif..."


In [75]:
print('The DataFrame shape is', df2.shape)

The DataFrame shape is (103, 3)
